<a href="https://colab.research.google.com/github/SamuelFiq/RAG_TOCKALL/blob/main/Llama_2_RAG_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RAG with LLaMa 13B

In this notebook we'll explore how we can use the open source **Llama-13b-chat** model in both Hugging Face transformers and LangChain.
At the time of writing, you must first request access to Llama 2 models via [this form](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) (access is typically granted within a few hours). If you need guidance on getting access please refer to the beginning of this [article](https://www.pinecone.io/learn/llama-2/) or [video](https://youtu.be/6iHVJyX2e50?t=175).

---

🚨 _Note that running this on CPU is sloooow. If running on Google Colab you can avoid this by going to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**. This should be included within the free tier of Colab._

---

We start by doing a `pip install` of all required libraries.

In [ ]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  pinecone-client==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.1 MB/s 

## Initializing the Hugging Face Embedding Pipeline

We begin by initializing the embedding pipeline that will handle the transformation of our docs into vector embeddings. We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding.

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

We can use the embedding model to create document embeddings like so:

In [ ]:
docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 2 doc embeddings, each with a dimensionality of 384.


## Building the Vector Index

We now need to use the embedding pipeline to build our embeddings and store them in a Pinecone vector index. To begin we'll initialize our index, for this we'll need a [free Pinecone API key](https://app.pinecone.io/).

In [ ]:
import os
import pinecone

# get API key from app.pinecone.io and environment from console
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY') or 'be227b3a-d248-411d-8808-00b17f911811',
    environment=os.environ.get('PINECONE_ENVIRONMENT') or 'us-east1-gcp-free'
)

Now we initialize the index.

In [ ]:
import time

index_name = 'llama-2-rag'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(embeddings[0]),
        metric='cosine'
    )
    # wait for index to finish initialization
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

MaxRetryError: HTTPSConnectionPool(host='controller.us-east1-gcp-free.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f63e24aca30>: Failed to resolve 'controller.us-east1-gcp-free.pinecone.io' ([Errno -2] Name or service not known)"))

Now we connect to the index:

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

MaxRetryError: HTTPSConnectionPool(host='llama-2-rag-unknown.svc.us-east1-gcp-free.pinecone.io', port=443): Max retries exceeded with url: /describe_index_stats (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f640c854520>: Failed to resolve 'llama-2-rag-unknown.svc.us-east1-gcp-free.pinecone.io' ([Errno -2] Name or service not known)"))

With our index and embedding process ready we can move onto the indexing process itself. For that, we'll need a dataset. We will use a set of Arxiv papers related to (and including) the Llama 2 research paper.

In [ ]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/llama-2-arxiv-papers-chunked',
    split='train'
)
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

We will embed and index the documents like so:

In [ ]:
data = data.to_pandas()

batch_size = 32

for i in range(0, len(data), batch_size):
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    texts = [x['chunk'] for i, x in batch.iterrows()]
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

MaxRetryError: HTTPSConnectionPool(host='llama-2-rag-unknown.svc.us-east1-gcp-free.pinecone.io', port=443): Max retries exceeded with url: /vectors/upsert (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f64072d02e0>: Failed to resolve 'llama-2-rag-unknown.svc.us-east1-gcp-free.pinecone.io' ([Errno -2] Name or service not known)"))

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.04288,
 'namespaces': {'': {'vector_count': 4288}},
 'total_vector_count': 4288}

## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `meta-llama/Llama-2-13b-chat-hf`.

* The respective tokenizer for the model.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = '*******************************'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The Llama 2 13B models were trained using the Llama 2 13B tokenizer, which we initialize like so:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Confirm this is working:

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

Explain to me the difference between nuclear fission and fusion. Unterscheidung zwischen Nuklearfusion und -fission.
Nuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This is typically achieved through the use of neutron bombardment, where a nucleus is struck by a neutron, causing it to become unstable and split into two or more smaller nuclei. Fission reactions are typically used in nuclear reactors to generate electricity, as the released energy can be harnessed to produce steam, which then drives a turbine to generate electricity.
Nuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but in this case, the energy is produced through the binding of the nuclei together. Fusion reactions are typically used in the sun and other stars, where they are responsible for producing the light and heat that we s

Now to implement this in LangChain

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
llm(prompt="Explain to me the difference between nuclear fission and fusion.")

' Unterscheidung zwischen Nuklearfusion und -fission.\nNuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This is typically achieved through the use of neutron bombardment, where a nucleus is struck by a neutron, causing it to become unstable and split into two or more smaller nuclei. Fission reactions are typically used in nuclear reactors to generate electricity, as the released energy can be harnessed to produce steam, which then drives a turbine to generate electricity.\nNuclear fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but in this case, the energy is produced through the binding of the nuclei together. Fusion reactions are typically used in the sun and other stars, where they are responsible for producing the light and heat that we see.\nThe main difference between nuclear fission and fusion i

We still get the same output as we're not really doing anything differently here, but we have now added **Llama 2 13B Chat** to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with **Llama 2**.

## Initializing a RetrievalQA Chain

For **R**etrieval **A**ugmented **G**eneration (RAG) in LangChain we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object. For both of these we need an `llm` (which we have initialized) and a Pinecone index — but initialized within a LangChain vector store object.

Let's begin by initializing the LangChain vector store, we do it like so:

In [ ]:
from langchain.vectorstores import Pinecone

text_field = 'text'  # field in metadata that contains text content

vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

We can confirm this works like so:

In [ ]:
query = 'what makes llama 2 special?'

vectorstore.similarity_search(
    query,  # the search query
    k=3  # returns top 3 most relevant chunks of text
)

[Document(page_content='Ricardo Lopez-Barquilla, Marc Shedroﬀ, Kelly Michelena, Allie Feinstein, Amit Sangani, Geeta\nChauhan,ChesterHu,CharltonGholson,AnjaKomlenovic,EissaJamil,BrandonSpence,Azadeh\nYazdan, Elisa Garcia Anzano, and Natascha Parks.\n•ChrisMarra,ChayaNayak,JacquelinePan,GeorgeOrlin,EdwardDowling,EstebanArcaute,Philomena Lobo, Eleonora Presani, and Logan Kerr, who provided helpful product and technical organization support.\n46\n•Armand Joulin, Edouard Grave, Guillaume Lample, and Timothee Lacroix, members of the original\nLlama team who helped get this work started.\n•Drew Hamlin, Chantal Mora, and Aran Mun, who gave us some design input on the ﬁgures in the\npaper.\n•Vijai Mohan for the discussions about RLHF that inspired our Figure 20, and his contribution to the\ninternal demo.\n•Earlyreviewersofthispaper,whohelpedusimproveitsquality,includingMikeLewis,JoellePineau,\nLaurens van der Maaten, Jason Weston, and Omer Levy.', metadata={'source': 'http://arxiv.org/pdf/230

Looks good! Now we can put our `vectorstore` and `llm` together to create our RAG pipeline.

In [ ]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=vectorstore.as_retriever()
)

Let's begin asking questions! First let's try *without* RAG:

In [ ]:
llm('what is so special about llama 2?')

"\n Unterscheidung between llama and llama 2 is not straightforward, as both are similar in appearance and have similar features. However, there are some key differences that set them apart:\n\n1. Design: Llama 2 has a more modern and sleek design compared to the original Llama. The new app has a cleaner and more minimalistic look, with a focus on simplicity and ease of use.\n2. Features: Llama 2 offers more advanced features than its predecessor, including the ability to create and manage multiple social media accounts from a single dashboard, as well as improved analytics and reporting tools.\n3. User interface: The user interface of Llama 2 is more intuitive and user-friendly than the original Llama, with a focus on making it easy for users to navigate and manage their social media accounts.\n4. Integration: Llama 2 offers better integration with other social media platforms, allowing users to easily share content across different networks and manage their online presence in a more 

Hmm, that's not what we meant... What if we use our RAG pipeline?

In [ ]:
rag_pipeline('what is so special about llama 2?')

{'query': 'what is so special about llama 2?',
 'result': ' Llama 2 is a collection of large language models (LLMs) that have been developed and released by Meta AI. These models range in scale from 7 billion to 70 billion parameters and are optimized for dialogue use cases. The authors claim that their ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc, outperform open-source chat models on most benchmarks they tested and provide a detailed description of their approach to ﬁne-tuning and safety. They also mention that their models enable interaction with humans through intuitive chat interfaces, which has led to rapid and widespread adoption among the general public.'}

This looks *much* better! Let's try some more.

In [ ]:
llm('what safety measures were used in the development of llama 2?')

"\n nobody knows.\n\nBut I can tell you that the llama 2 was developed by a team of experienced software developers who have a proven track record of creating high-quality, secure software. They used a variety of techniques and tools to ensure that the llama 2 was as safe and secure as possible, including:\n\n* Code reviews: The development team thoroughly reviewed each other's code to identify any potential security vulnerabilities.\n* Testing: The team conducted extensive testing to ensure that the llama 2 functioned correctly and did not contain any security flaws.\n* Security audits: Independent security experts conducted regular security audits to identify any potential weaknesses in the llama 2.\n* Penetration testing: The team simulated attacks on the llama 2 to identify any potential vulnerabilities and fix them before they could be exploited by attackers.\n\nOverall, the development team took a comprehensive approach to ensuring the security and safety of the llama 2, using a 

Okay, it looks like the LLM with no RAG is less than ideal — let's stop embarassing the poor LLM and stick with RAG + LLM. Let's ask the same question to our RAG pipeline.

In [ ]:
rag_pipeline('what safety measures were used in the development of llama 2?')

{'query': 'what safety measures were used in the development of llama 2?',
 'result': ' The safety measures used in the development of Llama 2 include:\n\n* Ethical considerations and limitations: We considered the ethical implications of developing a language model and took steps to mitigate any potential risks.\n* Responsible release strategy: We developed a responsible release strategy that includes releasing the model under a license and providing an acceptable use policy for users.\n* Safety tuning: We performed safety tuning to ensure that the model does not produce inaccurate or objectionable responses to user prompts.\n* Design input: We received design input from early reviewers of the paper to improve the quality of the figures in the paper.\n* Red teaming: We delayed the release of the 34B model due to a lack of time to sufficiently red team the model.\n* Publicly available resources: We used publicly available online sources for pretraining the model.\n* Safety testing and 

A reasonable answer from the RAG pipeline, but it doesn't contain much information — maybe we can ask more about this, like what is this _"red team"_ procedure that delayed the launch of the 34B model?

In [ ]:
rag_pipeline('what red teaming procedures were followed for llama 2?')

{'query': 'what red teaming procedures were followed for llama 2?',
 'result': ' According to the paper, the authors followed a responsible release strategy and delayed the release of the 34B model due to a lack of time to sufficiently red team. They also mention that they performed multiple rounds of red teaming over several months to measure the robustness of each new model as it was released internally. Additionally, they devised a metric called "robustness" to quantify the model\'s ability to resist violating responses triggered by red teaming exercises executed by a set of experts.'}

Very interesting!

In [ ]:
rag_pipeline('how does the performance of llama 2 compare to other local LLMs?')

{'query': 'how does the performance of llama 2 compare to other local LLMs?',
 'result': " The paper provides a comparison of the performance of Llama 2 with other local LLMs in terms of token sampling latency and human evaluation scores. According to the paper, Llama 2 achieves lower token sampling latency than other local LLMs on 16 TPU v4s, while providing similar or better human evaluation scores. Specifically, Llama 2 achieves a mean token sampling latency of 14.1ms on 16 TPU v4s, which is faster than the next best local LLM by 19%. Additionally, Llama 2 performs similarly or better than other local LLMs on human evaluation tasks, such as ROUGE-2 and human evaluation (100 shot).\n\nUnhelpful Answer: I don't know the answer to your question because I don't have access to the specific information you are looking for."}